
# Minitarea 5


-----------------

Nombre: 

Fecha de Entrega:


## Instrucciones

- El ejercicio consiste en:

    - Responder preguntas relativas a los contenidos vistos en los vídeos y slides de las clases. 
    
    - Utilizar la librería Transformers

- La minitarea es INDIVIDUAL.

- Está demás decir que no se admiten copias, ni de código, ni de respuestas escritas. 

- La entrega debe ser por u-cursos.

- Atrasos: se descontará un punto por día hábil de atraso tanto para las mini-tareas como para las competencias.

- En el horario de auxiliar se abrirán horarios de consulta en donde podrán preguntar acerca del ejercicio y en general, de todo el curso. 

- Cada sección tiene un punto base y se evalúa sobre 6 puntos.

- Al revisar, tu código será ejecutado. Verifica que tu entrega no tenga errores.

## Preguntas Teóricas
Para estas preguntas no es necesario implementar código, pero pueden utilizar pseudo código.

### Arquitecturas de Redes Neuronales

**Pregunta 1**: Explique el principal problema de las redes Elman recurrentes. Explique cada compuerta de las redes LSTM y la GRU.  (1,5 puntos)

**Respuesta**:

El problema de las redes Elman (o S-RNN) es el problema del vanishing gradient (o gradiente desvaneciente). Este problema se da porque los gradientes de las primeras capas tienden a 0 y puede hacer que estas capas no aprendan nada.

Para la LSTM: (0.5 ptos)
Cada compuerta es una combinación lineal entre el input $\vec x_j$ y el estado anterior $\vec h_{j-1}$ pasados por una tanh.
- $\vec i$: Que cosas me importan del input
- $\vec f$: Que cosas olvido de $\vec c_j$
- $\vec o$: Que es lo que me importa para producir un output

Para la GRU: (0.5 puntos)
- $\vec r$: Se utiliza para controlar el acceso al estado previo y proponer un candidato $\vec{\tilde s_j}$
- $\vec z$: Genera una interpolación entre el estado anterior y el candidato de $\vec r$


**Pregunta 2**: Explique cuales son las diferencias entre las tres arquitecturas de sequence to sequence vistas en clases (Encoder-Decoder con RNN, Encoder-Decoder con RNN y Attention, y el Transformer) (1,5 puntos)

**Respuesta**: 

Attention ayuda al Decoder a aprender a resaltar las partes importantes del Encoder. El Transformer permite una paralelización superior a las RNN's.

**Pregunta 3**: ¿Cúal es el principal diferencia entre los Embeddings contextualizados (por ejemplo BERT) vs. los Embeddings estáticos (por ejemplo Word2Vec)? (1,5 puntos)

**Respuesta**: 

Los embeddings contextualizados le dan una representación a cada palabra que depende del contexto en donde aparece. Una palabra tiene tantas representaciones como contextos. Las palabras tienen solo una representación en los Embeddings estaticos, la cual fue aprendida durante el entrenamiento.



**Pregunta 4**: Explique en que tareas y las arquitecturas con las que se entrenan ELMO y BERT (1,5 puntos)

**Respuesta**: 
- ELMO es un Language Model usando RNN.  
- BERT hace masked language modeling (eliminan una palabra de una oración y debe adivinarla) y Next Sentence Prediction (Una oración A se le dan dos posibles opciones para continuar y BERT debe predecir cúal es, B o C, en la cual una de las dos es la continuación más factible) usando un Transformer Encoder.

## Preguntas Prácticas:

### BERT

Lo primero es instalar las librerías necesarias.

In [ ]:
!pip install transformers
from transformers import BertTokenizer, BertForNextSentencePrediction, BertForMaskedLM, BertForQuestionAnswering
import torch

     |████████████████████████████████| 778kB 4.6MB/s 
     |████████████████████████████████| 3.0MB 13.9MB/s 
     |████████████████████████████████| 1.1MB 35.5MB/s 
     |████████████████████████████████| 890kB 47.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=19b0534dc1edb793786cb5ed14a984e4317c06f5026336c862353a7a3ea39406
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Para las preguntas que siguen, utilizaremos distintas variantes de BERT disponibles en la librería transformers. [Aquí](https://huggingface.co/transformers/model_doc/bert.html) pueden encontrar toda la documentación necesaria. El modelo pre-entrenado a utilizar es "bert-base-uncased" (salvo para question answering).

BERT es un modelo de lenguaje que fue entrenado exhaustivamente sobre dos tareas: 1) Next sentence prediction. 2) Masked language modeling. Veremos ejemplos de esto en la Auxiliar del jueves 16/07/20

#### BertForNextSentencePrediction

**Pregunta 1** (2 puntos en total): Utilizando el modelo BertForNextSentencePrediction de la librería transformers, muestre cual de las 2 oraciones es más probable que sea una continuación de la primera. (0.5 puntos cada una)

Por ejemplo:

Initial: "The sky is blue."\
A: "This is due to the shorter wavelength of blue light."\
B: "Chile is one of the world's greatest economies."

Debería retornar "La oración que continúa más probable es A", justificándolo con la evaluación de BERT.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Funciones auxiliares:
def oracion_mas_probable(first,sentA,sentB):
  encodingA = tokenizer(first, sentA, return_tensors='pt')
  encodingB = tokenizer(first, sentB, return_tensors='pt')
  lossA, logitsA = model(**encodingA, next_sentence_label=torch.LongTensor([1]))
  lossB, logitsB = model(**encodingB, next_sentence_label=torch.LongTensor([1]))
  #Nota logits[0,0] entrega el score que la oracion si sea la siguiente (que tan True)
  #logits[0,1] entrega el score de que la oracion no sea la siguiente (que tan False)
  # Se puede aplicar una SoftMax sobre estos resultados para que sean probabilidades
  # Pero no es necesario.
  if logitsA[0, 0] > logitsB[0, 0] and logitsA[0, 1] < logitsB[0, 1]:
    print("La oración más probable es A")
  elif logitsB[0, 0] > logitsA[0, 0] and logitsB[0, 1] < logitsA[0, 1]:
    print("La oración más probable es B")
  else:
    print("No está claro cual es más probable")

1.1)
Initial: "My cat is fluffy."\
A: "My dog has a curling tail."\
B: "A song can make or ruin a person’s day if they let it get to them."

In [ ]:
initial = "My cat is fluffy."
sentenceA = "My dog has a curly tail."
sentenceB = "A song can make or ruin a person’s day if they let it get to them."
oracion_mas_probable(initial,sentenceA,sentenceB)

La oración más probable es A


1.2)
Initial: "The Big Apple is famous worldwide."\
A: "You can add cinnamon for the perfect combination."\
B: "It is America's largest city."

In [ ]:
initial = "The Big Apple is famous worldwide."
sentenceA = "You can add cinnamon for the perfect combination."
sentenceB = "It is America's largest city."
oracion_mas_probable(initial,sentenceA,sentenceB)

La oración más probable es B


1.3)
Initial: "Roses are red."\
A: "Violets are blue."\
B: "Fertilize them regularly for impressive flowers."

In [ ]:
initial = "Roses are red."
sentenceA = "Violets are blue."
sentenceB = "Fertilize them regularly for impressive flowers."
oracion_mas_probable(initial,sentenceA,sentenceB)

La oración más probable es A


1.4)
Initial: "I play videogames the whole day."\
A: "They make me happy."\
B: "They make me rage."\
<sub><sup><sub><sup>No estoy tan de acuerdo :D

In [ ]:
initial = "I play videogames the whole day."
sentenceA = "They make me happy."
sentenceB = "They make me rage."
oracion_mas_probable(initial,sentenceA,sentenceB)

La oración más probable es A


####BertForMaskedLM

**Pregunta 2** (2 puntos en total): Ahora utilizaremos BertForMaskedLM para predecir una palabra oculta en una oración. (0.5 puntos cada una)\
Por ejemplo:\
BERT input: "I want to _ a new car."\
BERT prediction: "buy"

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Funcion auxiliar
def palabra_mas_probable(sentence):
  tokenized_text = tokenizer.tokenize(sentence)
  masked_index = tokenized_text.index('[MASK]')
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_ids = [0] * len(tokenized_text)
  segments_tensors = torch.tensor([segments_ids])
  predictions = model(tokens_tensor, segments_tensors)
  predicted_index = torch.argmax(predictions[0][0][masked_index]).item()
  predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
  print(predicted_token)

2.1)
BERT input: "[CLS] I love [MASK] . [SEP]"

In [ ]:
sent = "[CLS] I love [MASK] . [SEP]"
palabra_mas_probable(sent)

you


2.2)
BERT input: "[CLS] I hear that Karen is very [MASK] . [SEP]"

In [ ]:
sent = "[CLS] I heard that Karen is very [MASK] . [SEP]"
palabra_mas_probable(sent)

upset


2.3)
BERT input: "[CLS] She had the gift of being able to [MASK] . [SEP]"

In [ ]:
sent = "[CLS] She had the gift of being able to [MASK] . [SEP]"
palabra_mas_probable(sent)

fly


2.4)
BERT input: "[CLS] It's not often you find a [MASK] on the street. [SEP]"

In [ ]:
sent = "[CLS] It's not often you find an [MASK] on the circus . [SEP]"
palabra_mas_probable(sent)

expert


####BertForQuestionAnswering

**Pregunta 3**: (2 puntos) Utilizando el modelo BertForQuestionAnswering pre-entrenado con 'bert-large-uncased-whole-word-masking-finetuned-squad', extraiga la respuesta a cada una de las siguientes 4 preguntas y su contexto. (0.5 puntos cada una). Recuerde cambiar el tokenizer para que coincida con el modelo.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
# Funcion Auxiliar
def entregar_respuesta(qst, cntxt):
  input_text = qst + " [SEP] " + cntxt
  input_ids = tokenizer.encode(input_text)
  token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]
  start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
  all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
  print(' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))

3.1)

Pregunta: "When was the Battle of Iquique?"

Contexto: "The Battle of Iquique was a naval engagement that occurred between a Chilean corvette under the command of Arturo Prat and a Peruvian ironclad under the command of Miguel Grau Seminario on 21 May 1879, during the naval stage of the War of the Pacific, and resulted in a Peruvian victory."

In [ ]:
q = "When was the Battle of Iquique?"
c = "The Battle of Iquique was a naval engagement that occurred between a Chilean corvette under the command of Arturo Prat and a Peruvian ironclad under the command of Miguel Grau Seminario on 21 May 1879, during the naval stage of the War of the Pacific, and resulted in a Peruvian victory."
entregar_respuesta(q, c)

21 may 1879


3.2)

Pregunta: "Who won the Battle of Iquique?"

Contexto: "The Battle of Iquique was a naval engagement that occurred between a Chilean corvette under the command of Arturo Prat and a Peruvian ironclad under the command of Miguel Grau Seminario on 21 May 1879, during the naval stage of the War of the Pacific, and resulted in a Peruvian victory."

In [ ]:
q = "Who won the Battle of Iquique?"
c = "The Battle of Iquique was a naval engagement that occurred between a Chilean corvette under the command of Arturo Prat and a Peruvian ironclad under the command of Miguel Grau Seminario on 21 May 1879, during the naval stage of the War of the Pacific, and resulted in a Peruvian victory."
entregar_respuesta(q, c)

peruvian


3.3)

Pregunta: "Who introduced peephole connections to LSTM networks?"
Contexto: "What I’ve described so far is a pretty normal LSTM. But not all LSTMs are the same as the above. In fact, it seems like almost every paper involving LSTMs uses a slightly different version. The differences are minor, but it’s worth mentioning some of them. One popular LSTM variant, introduced by Gers & Schmidhuber (2000), is adding “peephole connections.” This means that we let the gate layers look at the cell state."

In [ ]:
q = "Who introduced peephole connections to LSTM networks?"
c = "What I’ve described so far is a pretty normal LSTM. But not all LSTMs are the same as the above. In fact, it seems like almost every paper involving LSTMs uses a slightly different version. The differences are minor, but it’s worth mentioning some of them. One popular LSTM variant, introduced by Gers & Schmidhuber (2000), is adding “peephole connections.” This means that we let the gate layers look at the cell state."
entregar_respuesta(q, c)

ge ##rs & sc ##hmi ##dh ##uber


3.4)

Pregunta: "When is the cat most active?"

Contexto: "The cat is similar in anatomy to the other felid species: it has a strong flexible body, quick reflexes, sharp teeth and retractable claws adapted to killing small prey. Its night vision and sense of smell are well developed. Cat communication includes vocalizations like meowing, purring, trilling, hissing, growling and grunting as well as cat-specific body language. It is a solitary hunter but a social species. It can hear sounds too faint or too high in frequency for human ears, such as those made by mice and other small mammals. It is a predator that is most active at dawn and dusk. It secretes and perceives pheromones."

In [ ]:
q = "When is the cat most active?"
c = "The cat is similar in anatomy to the other felid species: it has a strong flexible body, quick reflexes, sharp teeth and retractable claws adapted to killing small prey. Its night vision and sense of smell are well developed. Cat communication includes vocalizations like meowing, purring, trilling, hissing, growling and grunting as well as cat-specific body language. It is a solitary hunter but a social species. It can hear sounds too faint or too high in frequency for human ears, such as those made by mice and other small mammals. It is a predator that is most active at dawn and dusk. It secretes and perceives pheromones."
entregar_respuesta(q, c)

dawn and dusk
